In [1]:
# import packages
import gurobipy as gp
from gurobipy import GRB
import sensitivity_analysis as sa

In [2]:
# Set up the data
prod_cities = ['ft_collins', 'provo']
demand_cities = ['denver', 'omaha', 'seattle']

capacity = {'ft_collins': 24,
            'provo': 27}
demand = {'denver': 6,
          'omaha': 7,
          'seattle': 11}

costs = {'ft_collins': {'denver': 100,
                        'omaha': 495,
                        'seattle': 1200},
         'provo': {'denver': 500,
                   'omaha': 950,
                   'seattle': 800}}

In [3]:
# Create the model
cc = gp.Model('culinary_compacts')
cc.ModelSense = GRB.MINIMIZE

Set parameter Username
Set parameter LicenseID to value 2620317
Academic license - for non-commercial use only - expires 2026-02-10


In [4]:
# Create decision variables
dvars = cc.addVars(prod_cities, demand_cities, vtype=GRB.CONTINUOUS, lb=0.0, name='x')
cc.update()
dvars

{('ft_collins', 'denver'): <gurobi.Var x[ft_collins,denver]>,
 ('ft_collins', 'omaha'): <gurobi.Var x[ft_collins,omaha]>,
 ('ft_collins', 'seattle'): <gurobi.Var x[ft_collins,seattle]>,
 ('provo', 'denver'): <gurobi.Var x[provo,denver]>,
 ('provo', 'omaha'): <gurobi.Var x[provo,omaha]>,
 ('provo', 'seattle'): <gurobi.Var x[provo,seattle]>}

In [5]:
# Set the objective function
cc.setObjective(gp.quicksum(costs[i][j]*dvars[i,j]
                            for i in prod_cities
                            for j in demand_cities))
cc.update()
cc.display()

Minimize
100.0 x[ft_collins,denver] + 495.0 x[ft_collins,omaha] + 1200.0 x[ft_collins,seattle]
+ 500.0 x[provo,denver] + 950.0 x[provo,omaha] + 800.0 x[provo,seattle]
Subject To


C:\Users\mddean\AppData\Local\Temp\ipykernel_21532\657226575.py:6: DeprecationWarning: Model.display() is deprecated
  cc.display()


In [6]:
# Constraints
# Capacity constraints
for i in prod_cities:
    cc.addLConstr(gp.quicksum(dvars[i,j] for j in demand_cities),
                  GRB.LESS_EQUAL,
                  rhs=capacity[i],
                  name=f'capacity_{i}')

# Demand constraints
for j in demand_cities:
    cc.addLConstr(gp.quicksum(dvars[i,j] for i in prod_cities),
                  GRB.GREATER_EQUAL,
                  rhs=demand[j],
                  name=f'demand_{j}')

cc.update()
cc.display()

Minimize
100.0 x[ft_collins,denver] + 495.0 x[ft_collins,omaha] + 1200.0 x[ft_collins,seattle]
+ 500.0 x[provo,denver] + 950.0 x[provo,omaha] + 800.0 x[provo,seattle]
Subject To
capacity_ft_collins: x[ft_collins,denver] + x[ft_collins,omaha] + x[ft_collins,seattle]
 <= 24
  capacity_provo: x[provo,denver] + x[provo,omaha] + x[provo,seattle] <= 27
  demand_denver: x[ft_collins,denver] + x[provo,denver] >= 6
  demand_omaha: x[ft_collins,omaha] + x[provo,omaha] >= 7
  demand_seattle: x[ft_collins,seattle] + x[provo,seattle] >= 11


C:\Users\mddean\AppData\Local\Temp\ipykernel_21532\3113065830.py:17: DeprecationWarning: Model.display() is deprecated
  cc.display()


In [7]:
# optimize
cc.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 7 155H, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 22 logical processors, using up to 22 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xfe8059f6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 3e+01]
Presolve time: 0.01s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.400000e+01   0.000000e+00      0s
       3    1.2865000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.286500000e+04


In [8]:
sa.sa_vars(cc.getVars())

,final_value,reduced_cost,obj_coef,range_opt_low,range_opt_high
"x[ft_collins,denver]",6.0,0.0,100.0,0.0,500.0
"x[ft_collins,omaha]",7.0,0.0,495.0,0.0,950.0
"x[ft_collins,seattle]",0.0,400.0,1200.0,800.0,inf
"x[provo,denver]",0.0,400.0,500.0,100.0,inf
"x[provo,omaha]",0.0,455.0,950.0,495.0,inf
"x[provo,seattle]",11.0,0.0,800.0,0.0,1200.0


In [9]:
sa.sa_constrs(cc.getConstrs())

,binding?,final_value,RHS,slack,shadow_price,range_feas_low,range_feas_high
capacity_ft_collins,non-binding,13.0,24.0,11.0,0.0,13.0,inf
capacity_provo,non-binding,11.0,27.0,16.0,0.0,11.0,inf
demand_denver,binding,6.0,6.0,0.0,100.0,-0.0,17.0
demand_omaha,binding,7.0,7.0,0.0,495.0,-0.0,18.0
demand_seattle,binding,11.0,11.0,0.0,800.0,-0.0,27.0
